# apply() functions on a column

### Calculate Projected salary for next year

In [80]:
# import libraries & dataset
from datasets import load_dataset
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ast
dataset = load_dataset('lukebarousse/data_jobs')
df = dataset['train'].to_pandas()

# format job_posted date in datetime (it was string)
df.job_posted_date = pd.to_datetime(df.job_posted_date)


In [3]:
df[pd.notna(df['salary_year_avg'])]['salary_year_avg']

28        109500.0
77        140000.0
92        120000.0
100       228222.0
109        89000.0
            ...   
785624    139216.0
785641    150000.0
785648    221875.0
785682    157500.0
785692    157500.0
Name: salary_year_avg, Length: 22003, dtype: float64

In [4]:
help(df.apply)

Help on method apply in module pandas.core.frame:

apply(func: 'AggFuncType', axis: 'Axis' = 0, raw: 'bool' = False, result_type: "Literal['expand', 'reduce', 'broadcast'] | None" = None, args=(), by_row: "Literal[False, 'compat']" = 'compat', engine: "Literal['python', 'numba']" = 'python', engine_kwargs: 'dict[str, bool] | None' = None, **kwargs) method of pandas.core.frame.DataFrame instance
    Apply a function along an axis of the DataFrame.
    
    Objects passed to the function are Series objects whose index is
    either the DataFrame's index (``axis=0``) or the DataFrame's columns
    (``axis=1``). By default (``result_type=None``), the final return type
    is inferred from the return type of the applied function. Otherwise,
    it depends on the `result_type` argument.
    
    Parameters
    ----------
    func : function
        Function to apply to each column or row.
    axis : {0 or 'index', 1 or 'columns'}, default 0
        Axis along which the function is applied:
 

In [8]:
# create a new filtered df without null values
df_salary = df[pd.notna(df['salary_year_avg'])].copy()

# create formula for salary inflation adjustment
def projected_salary(salary):
    return salary * 1.03

# apply function to the column
df_salary['salary_year_adjusted'] = df_salary['salary_year_avg'].apply(projected_salary)

df_salary[['salary_year_avg', 'salary_year_adjusted']]

,salary_year_avg,salary_year_adjusted
28,109500.0,112785.00
77,140000.0,144200.00
92,120000.0,123600.00
100,228222.0,235068.66
109,89000.0,91670.00
...,...,...
785624,139216.0,143392.48
785641,150000.0,154500.00
785648,221875.0,228531.25
785682,157500.0,162225.00


In [ ]:
# alternatively usign a lambda function to make the function anonymous
df_salary['salary_year_adjusted'] = df_salary['salary_year_avg'].apply(lambda salary: salary * 1.03)

df_salary[['salary_year_avg', 'salary_year_adjusted']]

,salary_year_avg,salary_year_adjusted
28,109500.0,112785.00
77,140000.0,144200.00
92,120000.0,123600.00
100,228222.0,235068.66
109,89000.0,91670.00
...,...,...
785624,139216.0,143392.48
785641,150000.0,154500.00
785648,221875.0,228531.25
785682,157500.0,162225.00


Now we want to convert the job_skill from string to list 

In [16]:
type(df['job_skills'][1])

str

In [34]:
import ast
type(ast.literal_eval(df['job_skills'][1]))

list

In [ ]:
#create a function to clean list
def clean_list(skill_list):
    return ast.literal_eval(skill_list)

df['job_skills'] = df['job_skills'].apply(clean_list) #error because of none values

ValueError: malformed node or string: None

In [ ]:
def clean_list(skill_list):
    if pd.notna(skill_list): # making the rule valid only on notna cells prevents errors
        return ast.literal_eval(skill_list)

df['job_skills'] = df['job_skills'].apply(clean_list) 

In [26]:
df['job_skills'][1]

['r', 'python', 'sql', 'nosql', 'power bi', 'tableau']

In [35]:
df['job_skills'] = df['job_skills'].apply(lambda skill_list: ast.literal_eval(skill_list) if pd.notna(skill_list) else skill_list)


In [38]:
type(df['job_skills'][1])

list

# apply() functions on a row

### Calculate projected salary
- senior roles assume 5%
- other roles assume 3%

In [44]:
# defining formula for salary adjusment based on the role 
def projected_salary(row):
    if "Senior" in row['job_title_short']:
        return row['salary_year_avg'] * 1.05
    else:
        return row['salary_year_avg'] * 1.03

# create a column ad hoc
df_salary['salary_year_inflated'] = df_salary.apply(projected_salary, axis=1) #axis =1 means that applies to specific rows
df_salary[['job_title_short', 'salary_year_avg', 'salary_year_inflated']]

,job_title_short,salary_year_avg,salary_year_inflated
28,Data Scientist,109500.0,112785.00
77,Data Engineer,140000.0,144200.00
92,Data Engineer,120000.0,123600.00
100,Data Scientist,228222.0,235068.66
109,Data Analyst,89000.0,91670.00
...,...,...,...
785624,Data Engineer,139216.0,143392.48
785641,Data Engineer,150000.0,154500.00
785648,Data Scientist,221875.0,228531.25
785682,Data Scientist,157500.0,162225.00


In [ ]:
# same formula applying lambda function
df_salary['salary_year_inflated'] = df_salary.apply(
    lambda row: row['salary_year_avg'] * 1.05 if "Senior" in row['job_title_short'] else row['salary_year_avg'] * 1.03,
    axis=1
)

df_salary[['job_title_short', 'salary_year_avg', 'salary_year_inflated']]


,job_title_short,salary_year_avg,salary_year_inflated
28,Data Scientist,109500.0,112785.00
77,Data Engineer,140000.0,144200.00
92,Data Engineer,120000.0,123600.00
100,Data Scientist,228222.0,235068.66
109,Data Analyst,89000.0,91670.00
...,...,...,...
785624,Data Engineer,139216.0,143392.48
785641,Data Engineer,150000.0,154500.00
785648,Data Scientist,221875.0,228531.25
785682,Data Scientist,157500.0,162225.00


### Exercises

🟩 Convert Date to String (2.10.1) - Problem
Pandas Applying Functions
Problem Statement:

Convert the job_posted_date column to a string format 'YYYY-MM-DD' and create a new column job_posted_date_str.

In [54]:
df['job_posted_date_str'] = df['job_posted_date'].apply(lambda date: datetime.strftime(date, "%d/%m/%y"))
df[['job_posted_date', 'job_posted_date_str']]

,job_posted_date,job_posted_date_str
0,2023-06-16 13:44:15,16/06/23
1,2023-01-14 13:18:07,14/01/23
2,2023-10-10 13:14:55,10/10/23
3,2023-07-04 13:01:41,04/07/23
4,2023-08-07 14:29:36,07/08/23
...,...,...
785736,2023-03-13 06:16:16,13/03/23
785737,2023-03-12 06:18:18,12/03/23
785738,2023-03-12 06:32:36,12/03/23
785739,2023-03-12 06:32:15,12/03/23


In [57]:
print(type(df['job_posted_date'][0]))
print(type(df['job_posted_date_str'][0]))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'str'>


🟨 Days Since Posted (2.10.2) - Problem
Pandas Applying Functions
Problem Statement:

Calculate the number of days since each job was posted. Create a new column days_since_posted that contains this value. Use the job_posted_date column.

Note: You need to import the datetime library and use the datetime module to get the current date using .now().

In [70]:
today = datetime.now() 
df['days_since_posting'] = df['job_posted_date'].apply(lambda date: (today - date).days)
df[['job_posted_date','days_since_posting']].head()

,job_posted_date,days_since_posting
0,2023-06-16 13:44:15,676
1,2023-01-14 13:18:07,829
2,2023-10-10 13:14:55,560
3,2023-07-04 13:01:41,658
4,2023-08-07 14:29:36,624


🟥 Salary Category (2.10.3) - Problem
Pandas Applying Functions
Problem Statement:

Create a copy of the DataFrame called df_filtered and drop the NaN values for salary_year_avg.
Then, create a new column salary_category that categorizes the salary_year_avg into three categories: 'Low' for salaries less than 60,000, 'Medium' for salaries between 60,000 and 100,000, and 'High' for salaries greater than 100,000.
Then show the df_filtered DataFrame and the salary_year_avg and salary_category columns.


In [104]:
df_filtered = df.copy()
df_filtered = df.dropna(subset='salary_year_avg')
df_filtered['salary_category'] = df_filtered['salary_year_avg'].apply(lambda salary: 
    "High" if salary >= 100_000 
    else ("Medium" if 60_000 <= salary <100_000 
    else "Low")
    )
df_filtered[['salary_year_avg', 'salary_category']].head(10)

,salary_year_avg,salary_category
28,109500.0,High
77,140000.0,High
92,120000.0,High
100,228222.0,High
109,89000.0,Medium
116,114000.0,High
146,129500.0,High
180,90250.0,Medium
212,157500.0,High
257,103128.0,High


In [103]:
def salary_cluster(salary):
    if salary < 60_000:
        return "Low"
    elif salary < 100_000:
        return "Medium"
    else:
        return "High"

df_filtered = df.copy()
df_filtered = df_filtered.dropna(subset=['salary_year_avg'])  # subset must be a list!
df_filtered['salary_category'] = df_filtered['salary_year_avg'].apply(salary_cluster)

df_filtered[['salary_year_avg', 'salary_category']].head(10)


,salary_year_avg,salary_category
28,109500.0,High
77,140000.0,High
92,120000.0,High
100,228222.0,High
109,89000.0,Medium
116,114000.0,High
146,129500.0,High
180,90250.0,Medium
212,157500.0,High
257,103128.0,High
